<a href="https://colab.research.google.com/github/ranjini-rao/Neural-Network_PharmaProbe/blob/main/Clean_Neural_Net_For_Group_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd

# Load the CSV file into a DataFrame
embedded_df = pd.read_csv('embedded_review.csv')

# Display the first few rows of the DataFrame to verify it was loaded correctly
embedded_df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,lengthReview,conditionCluster_label,drugNameCluster_label,...,758,759,760,761,762,763,764,765,766,767
0,95260,Guanfacine,ADHD,My son is halfway through his fourth week of I...,8.0,27-Apr-10,192.0,712.0,2.0,4.0,...,-0.416409,-0.364040,-0.036060,0.383963,0.176255,-0.147201,-0.243359,-0.541467,0.062160,0.049585
1,92703,Lybrel,Birth Control,I used to take another oral contraceptive whic...,5.0,14-Dec-09,17.0,708.0,9.0,6.0,...,-0.279307,-0.419729,-0.389261,0.328398,0.291834,-0.027217,-0.359790,-0.706709,0.047264,-0.017902
2,138000,Ortho Evra,Birth Control,This is my first time using any form of birth ...,8.0,3-Nov-15,10.0,428.0,9.0,4.0,...,-0.232733,-0.031823,-0.032784,0.188440,0.162272,0.363399,-0.096650,-0.693634,-0.024901,0.548486
3,35696,Buprenorphine naloxone,Opiate Dependence,Suboxone has completely turned my life around ...,9.0,27-Nov-16,37.0,669.0,0.0,2.0,...,-0.310564,-0.599643,-0.375174,0.309915,0.577983,0.051811,-0.184821,-0.710691,0.065533,0.371945
4,155963,Cialis,Benign Prostatic Hyperplasia,2nd day on 5mg started to work with rock hard ...,2.0,28-Nov-15,43.0,373.0,0.0,5.0,...,-0.247983,-0.438636,-0.037911,-0.030183,0.508780,0.064493,-0.205261,-0.527391,-0.101341,0.039573


In [16]:
len(embedded_df)

11316

In [17]:
#This Neural Net is just going to focus on classifying the Condition Cluster by the Review Embeddings
#This Neural Net is also going to use the dataframe that has not had dimensions reduced

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [18]:
# I am going to drop the columns that are not embeddings of the review and the condition_cluster

columns_to_drop = ['uniqueID', 'drugName', 'date', 'condition', 'review', 'lengthReview']

embedded_df.drop(columns=columns_to_drop, inplace=True)

In [19]:
#check the dataframe

embedded_df.head()

,rating,usefulCount,conditionCluster_label,drugNameCluster_label,0,1,2,3,4,5,...,758,759,760,761,762,763,764,765,766,767
0,8.0,192.0,2.0,4.0,-0.010977,0.010914,0.200967,-0.229490,-0.535286,0.012419,...,-0.416409,-0.364040,-0.036060,0.383963,0.176255,-0.147201,-0.243359,-0.541467,0.062160,0.049585
1,5.0,17.0,9.0,6.0,0.066320,0.189584,0.369006,-0.046920,-0.473988,-0.238288,...,-0.279307,-0.419729,-0.389261,0.328398,0.291834,-0.027217,-0.359790,-0.706709,0.047264,-0.017902
2,8.0,10.0,9.0,4.0,0.084101,-0.019134,0.294494,0.029783,-0.228783,0.170102,...,-0.232733,-0.031823,-0.032784,0.188440,0.162272,0.363399,-0.096650,-0.693634,-0.024901,0.548486
3,9.0,37.0,0.0,2.0,0.007820,0.207558,0.179105,-0.210057,-0.197015,0.104799,...,-0.310564,-0.599643,-0.375174,0.309915,0.577983,0.051811,-0.184821,-0.710691,0.065533,0.371945
4,2.0,43.0,0.0,5.0,-0.193177,0.360585,0.448292,-0.253824,-0.532782,0.085381,...,-0.247983,-0.438636,-0.037911,-0.030183,0.508780,0.064493,-0.205261,-0.527391,-0.101341,0.039573


In [20]:
#checking on why the values of the condition cluster look like a float

condition_value_counts = embedded_df['conditionCluster_label'].value_counts()
print(condition_value_counts)

0.0    4074
8.0    2100
2.0     955
1.0     865
7.0     777
9.0     752
4.0     736
5.0     356
3.0     353
6.0     347
Name: conditionCluster_label, dtype: int64


In [21]:
# When I start training the model I am getting an error messsage that NAN values are present
embedded_df = embedded_df.dropna(subset=['conditionCluster_label'])

In [22]:
# Split our preprocessed data into our features and target arrays
# Choosing randon_state = 30 to be able to re-create attempts later
# There is a lot of data, so I can train on 70%, setting test_size at .3
# This step is necessary for training the neural network to classify 10 categories
# This code is altered by chatgpt after I could not fix it on my own

from keras.utils import to_categorical

# Separate the target variable
target = embedded_df['conditionCluster_label']

# Verify the unique values in the target variable
print(target.unique())

# Extract features (excluding the target variable)
features = embedded_df.drop(columns=['conditionCluster_label'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=30)

# Convert target labels to one-hot encoded format
y_train_encoded = to_categorical(y_train, num_classes=10)
y_test_encoded = to_categorical(y_test, num_classes=10)

# Check the shapes of the one-hot encoded target labels
print("Shape of y_train_encoded:", y_train_encoded.shape)
print("Shape of y_test_encoded:", y_test_encoded.shape)

[2. 9. 0. 1. 3. 8. 5. 7. 4. 6.]
Shape of y_train_encoded: (7920, 10)
Shape of y_test_encoded: (3395, 10)


In [23]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [24]:
# Check the number of dimensions, make it a variable so it passes into

num_dimensions = embedded_df.shape[1]-1
print(num_dimensions)

771


In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Most of this is going to be the same as the HW assignment, but there are 10 classifications to predict now
# So the output layer has been adjusted

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="sigmoid", input_dim=num_dimensions))

# Second hidden layer
nn.add(Dense(units=16, activation='sigmoid'))

# Third hidden layer (example of adding an additional hidden layer)
nn.add(Dense(units=8, activation='sigmoid'))

#Fourth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=4, activation='ELU'))

#Fifth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=4, activation='leaky_relu'))

#Sixth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=4, activation='relu'))

#Seventh hidden layer (example of adding another hidden layer)
nn.add(Dense(units=4, activation='ELU'))

#Eighth hidden layer (example of adding another hidden layer)
nn.add(Dense(units=4, activation='relu'))

# Output layer
nn.add(Dense(units=10, activation='softmax'))  # 10 units for 10 classes, softmax activation

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                24704     
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
 dense_3 (Dense)             (None, 4)                 36        
                                                                 
 dense_4 (Dense)             (None, 4)                 20        
                                                                 
 dense_5 (Dense)             (None, 4)                 20        
                                                                 
 dense_6 (Dense)             (None, 4)                 2

In [26]:
# Compile the model - the loss function is categorical and not for binary classification

nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
 # Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train_encoded, epochs=10)

Epoch 1/10
248/248 [==============================] - 4s 4ms/step - loss: 2.2383 - accuracy: 0.3566
Epoch 2/10
248/248 [==============================] - 1s 4ms/step - loss: 2.1343 - accuracy: 0.3600
Epoch 3/10
248/248 [==============================] - 1s 2ms/step - loss: 2.0632 - accuracy: 0.3600
Epoch 4/10
248/248 [==============================] - 1s 2ms/step - loss: 2.0172 - accuracy: 0.3600
Epoch 5/10
248/248 [==============================] - 1s 2ms/step - loss: 1.9890 - accuracy: 0.3600
Epoch 6/10
248/248 [==============================] - 1s 2ms/step - loss: 1.9729 - accuracy: 0.3600
Epoch 7/10
248/248 [==============================] - 1s 2ms/step - loss: 1.9641 - accuracy: 0.3600
Epoch 8/10
248/248 [==============================] - 1s 2ms/step - loss: 1.9597 - accuracy: 0.3600
Epoch 9/10
248/248 [==============================] - 1s 2ms/step - loss: 1.9574 - accuracy: 0.3600
Epoch 10/10
248/248 [==============================] - 1s 2ms/step - loss: 1.9563 - accuracy: 0.3600

In [28]:
test_loss, test_accuracy = nn.evaluate(X_test_scaled, y_test_encoded)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

107/107 [==============================] - 1s 3ms/step - loss: 1.9458 - accuracy: 0.3602
Test Loss: 1.9457931518554688
Test Accuracy: 0.36023563146591187


In [34]:
#Using L2 Normalization
from tensorflow.keras import regularizers

# Define the model - deep neural net with L2 regularization
nn = Sequential()

# First hidden layer with L2 regularization
nn.add(Dense(units=512, activation="sigmoid", kernel_regularizer=regularizers.l2(0.01), input_dim=num_dimensions))

# Second hidden layer with L2 regularization
nn.add(Dense(units=512, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))

# Third hidden layer with L2 regularization
nn.add(Dense(units=256, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))

# Fourth hidden layer with L2 regularization
nn.add(Dense(units=256, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))

# Fifth hidden layer with L2 regularization
nn.add(Dense(units=256, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))

# Sixth hidden layer with L2 regularization
nn.add(Dense(units=128, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))

# Seventh hidden layer with L2 regularization
nn.add(Dense(units=64, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))

# Eighth hidden layer with L2 regularization
nn.add(Dense(units=64, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))

# Output layer
nn.add(Dense(units=10, activation='softmax'))

# Check the structure of the model
nn.summary()

# Compile the model
nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train_encoded, epochs=200)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 512)               395264    
                                                                 
 dense_33 (Dense)            (None, 512)               262656    
                                                                 
 dense_34 (Dense)            (None, 256)               131328    
                                                                 
 dense_35 (Dense)            (None, 256)               65792     
                                                                 
 dense_36 (Dense)            (None, 256)               65792     
                                                                 
 dense_37 (Dense)            (None, 128)               32896     
                                                                 
 dense_38 (Dense)            (None, 64)               

KeyboardInterrupt: 

In [30]:
test_loss, test_accuracy = nn.evaluate(X_test_scaled, y_test_encoded)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

107/107 [==============================] - 1s 3ms/step - loss: 1.7450 - accuracy: 0.4159
Test Loss: 1.744950771331787
Test Accuracy: 0.41590574383735657
